In [1]:
# prompt: download model to local


# -*- coding: utf-8 -*-
"""connect4_final_1

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1CCgmf1e6eGHhf6_3p7b5wg8If0scxkWo
"""

#import anvil.server
import numpy as np
import os
os.environ["TF_TRT_ALLOW_BUILD_FAILURE"] = "1"
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers


# Replace with your actual Anvil Uplink Key
#ANVIL_UPLINK_KEY = "server_YRMKNBRJCQVXQ7JPEMRYXMUL-BLGMXSSKTGUJGMP2"

# Connect to Anvil
#anvil.server.connect(ANVIL_UPLINK_KEY)

print("Connected to Anvil successfully!")

#@anvil.server.callable
def initialize_board():
    """Initialize an empty Connect 4 board."""
    return np.zeros((6, 7, 2), dtype=int).tolist()  # Convert to list for client-side compatibility

#@anvil.server.callable
def drop_token(board, column, player):
    """Drop a token into the specified column."""
    board = np.array(board)  # Convert list back to NumPy array

    for row in range(5, -1, -1):
        if board[row, column, 0] == 0:
            board[row, column, 0] = player
            return board.tolist()  # Convert back to list before returning

    return board.tolist()  # Return unchanged board if column is full

###################

import tensorflow as tf
from tensorflow.keras.models import load_model

# Define the custom TransformerBlock and Connect4Transformer classes
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model=64, num_heads=8, ff_dim=128, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.attn = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='gelu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs, training=False):
        attn_output = self.attn(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "d_model": 64,
            "num_heads": 8,
            "ff_dim": 128,
            "dropout": 0.1
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

class Connect4Transformer(tf.keras.Model):
    def __init__(self, d_model=64, num_heads=4, ff_dim=128, num_layers=3, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.input_projection = tf.keras.layers.Dense(d_model)
        self.row_embed = tf.keras.layers.Embedding(input_dim=6, output_dim=d_model // 2)
        self.col_embed = tf.keras.layers.Embedding(input_dim=7, output_dim=d_model // 2)
        self.transformer_blocks = [TransformerBlock(d_model, num_heads, ff_dim, dropout) for _ in range(num_layers)]
        self.column_attention = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=d_model)
        self.output_layer = tf.keras.layers.Dense(7, activation='softmax')

    def call(self, inputs, training=False):
        x = tf.reshape(inputs, (-1, 6, 7, 2))
        batch_size = tf.shape(x)[0]
        rows = tf.tile(tf.range(6)[None, :, None], [batch_size, 1, 7])
        cols = tf.tile(tf.range(7)[None, None, :], [batch_size, 6, 1])

        row_emb = self.row_embed(rows)
        col_emb = self.col_embed(cols)
        pos_encoding = tf.concat([row_emb, col_emb], axis=-1)

        x = self.input_projection(x)
        x += pos_encoding

        x = tf.reshape(x, (-1, 6 * 7, self.d_model))
        for transformer in self.transformer_blocks:
            x = transformer(x)

        column_queries = tf.tile(tf.range(7)[None, :, None], [batch_size, 1, self.d_model])
        context = self.column_attention(column_queries, x)

        return self.output_layer(context[:, :, 0])

###################

#@anvil.server.callable
def predict_move(board_state):
    """Receives board state from Anvil, returns AI's move."""

    # Load the model with custom objects
    # model = load_model('save.keras', custom_objects={'PositionalEncoding': PositionalEncoding})
    model = load_model('conv_8H.keras', custom_objects={'TransformerBlock': TransformerBlock, 'Connect4Transformer': Connect4Transformer})
    # model = load_model('gpt_tx.h5')

    # Convert board to model input shape
    input_board = np.array(board_state).reshape(1, 6, 7, 2)
    # print(input_board)
    # print(input_board.shape)

    # Predict best move
    predictions = model.predict(input_board)
    best_move = int(np.argmax(predictions[0].flatten()))

    return best_move


Connected to Anvil successfully!


In [2]:
board =initialize_board()
ai_type = "CNN"
current_player = 1

In [3]:
column = 4
board = drop_token(board, column, 1)

In [4]:
current_player = 2
ai_move = predict_move(board)
ai_move

ValueError: A total of 13 objects could not be loaded. Example error message for object <EinsumDense name=key, built=True>:

Layer 'key' expected 2 variables, but received 0 variables during loading. Expected: ['kernel', 'bias']

List of objects that could not be loaded:
[<EinsumDense name=key, built=True>, <EinsumDense name=attention_output, built=True>, <EinsumDense name=query, built=True>, <EinsumDense name=value, built=True>, <EinsumDense name=key, built=True>, <EinsumDense name=attention_output, built=True>, <EinsumDense name=query, built=True>, <EinsumDense name=value, built=True>, <EinsumDense name=key, built=True>, <EinsumDense name=attention_output, built=True>, <EinsumDense name=query, built=True>, <EinsumDense name=value, built=True>, <keras.src.optimizers.adam.Adam object at 0x119293950>]